In [ ]:
%matplotlib widget
from dolfin import *
import numpy as nump
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

plt.close("all")
set_log_active(False)

cmap2 = mpl.cm.get_cmap("spring")
cmap = mpl.cm.get_cmap("winter")

# Define the problem
Length = 50.0
nx = 200
nT = 900
mesh = IntervalMesh(nx, 0, Length)
P1 = FiniteElement('P', interval, 1)
element = MixedElement([P1, P1,P1])
V = FunctionSpace(mesh, element)

# Define the parameters
b = Constant(1.0)
T = 140
dt = T/nT
alphval = 0.039
cval = 0.6
# Define the initial condition
u0 = Expression(('0.7*exp(-0.1*pow(abs(x[0]-5),2))',0,0), degree=1)
u_n = interpolate(u0, V)
u = Function (V)
n,np, rho = split(u)
n_n,np_n, rho_n= split(u_n)
v_1, v_0,v_2 = TestFunctions(V)
sp = Constant((0.3,))
k = Constant((1,))

s_scal_pull = Constant(0.2)
s_scal = Constant(0.1)

sp = Constant((s_scal_pull,))
beta = Constant(0.022)
alpha = Constant(alphval)
D = Constant(0.02)
alphantonp = Constant(0.020)
alphanpton = Constant(0.2)


np_mid = 0.5*(np_n + np)
n_mid = 0.5*(n_n + n)
np_mid = np
n_mid = n
v_s = Constant((0.235,))


# Define the variational problem

F = n*v_1*dx - n_n*v_1*dx 
F += dt*dot(grad(n_mid),v_s)*v_1*dx

#Diffusion coefficient
F += D*dot(grad(n), grad(v_1))*dt*dx



F -=dt*s_scal*n*v_2*dx-(rho-rho_n)*v_2*dx+dt*s_scal_pull*np*v_2*dx
F += beta*v_1*rho*(n_mid)*dt*dx
F -= alpha*(n_mid)*v_1*dt*dx
F += alphantonp*(n_mid)*v_1*dt*dx

F += np*v_0*dx - (np_n)*v_0*dx 
F -= alphantonp*(n_mid)*v_0*dt*dx
F += alphanpton*(np)*v_0*dt*dx
# F += beta*v_0*rho*(np)*dt*dx

# F += 0.1*np*v_0*dt*dx
# Apply the boundary conditions
u_L = (Constant(0.0),Constant(0.0),Constant(0.0))
u_R = (Constant(0.0),Constant(0.0),Constant(0.0))
bc_L = DirichletBC(V, u_L, 'near(x[0], 0.0)')
bc_R = DirichletBC(V, u_R, f'near(x[0], {Length})')
bcs = [bc_L, bc_R]

# bc = PeriodicBC(V, Boundary())# Solve the problem
t = 0.0
a, L = lhs(F), rhs(F)
V2 = FunctionSpace(mesh, 'P', 1)
X = nump.linspace(0,Length,nx)
poss = []

fix,axs = plt.subplots(3,1)
i=0
for t in nump.linspace(0,T,nT):

    solve(F==0, u,bcs)
    u_n.assign(u)
    _n,_n_p, _rho= u.split()
    n_field = interpolate(_n, V2)
    if i%150==0:
        rho_field = interpolate(_rho, V2)
        np_field = interpolate(_n_p, V2)
        axs[0].plot(X,[n_field(x) for x in X],color=cmap2(t/T))
        
        # axs[0].set_yscale('log')
        axs[2].plot(X,[rho_field(x) for x in X],color=cmap(t/T))
        axs[1].plot(X,[np_field(x) for x in X],color=cmap2(t/T))
    poss.append(nump.argmax([n_field(x) for x in X]))
    i+=1
    # break
axs[0].set_xlabel('',size=14)
axs[0].set_ylabel('$n (mm^{-2})$')

axs[2].set_xlabel('position ($mm$)')
axs[1].set_ylabel('$n_{BAS}(mm^{-2})$')
axs[1].set_xlabel('',size=14)
axs[2].set_ylabel(r'$\rho (mm.mm^{-2})$')
    
path_figure = r"figures"

plt.savefig(os.path.join(path_figure,'FigureS7D.pdf'), transparent=True, bbox_inches='tight')

In [ ]:
x = nump.linspace(0,T,nT)
y = nump.array(poss)*Length/nx
fit_coeffs = nump.polyfit(x[3*nT//4:], y[3*nT//4:], 1)
fit_line = nump.polyval(fit_coeffs, x)

# Plot data and linear fit
fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, fit_line, 'r--')
ax.set_xlabel('t (h)')
ax.set_ylabel('x ($mm$)')

